In [1]:
import configparser
import psycopg2
import pandas as pd

In [2]:
import pandas.io.sql as sqlio
from IPython.display import display

In [3]:
%load_ext sql

In [4]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [5]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

In [6]:
conn.autocommit=True

**Some data-checks**

In [7]:
def get_count_for_table(table_name):
    count_query = f"""
    SELECT COUNT(*) AS count 
    FROM {table_name}
    """
    count = sqlio.read_sql_query(count_query, conn)
    return count['count'][0]

In [8]:
table_names = [('artist_dim','artist_id'),
              ('song_dim', 'song_id'),
              ('user_dim','user_id'),
              ('time_dim','start_time'),
              ('songplay_fact', 'songplay_id')
              ]
count_dict = dict()
for table in table_names:
    count_dict[table[0]] = get_count_for_table(table[0])

In [9]:
count_df = pd.DataFrame.from_dict(count_dict, orient='index').reset_index()
count_df= count_df.rename(columns={"index": "table_name", 0: "count"})

In [10]:
print("Number of rows in tables:")
count_df

Number of rows in tables:


,table_name,count
0,artist_dim,10025
1,song_dim,14896
2,user_dim,104
3,time_dim,8023
4,songplay_fact,333


In [11]:
def get_distinct_count_for_table(table_name, pkey):
    distinct_count_query = f"""
    SELECT COUNT(DISTINCT({pkey})) AS distinct_count 
    FROM {table_name}
    """
    count = sqlio.read_sql_query(distinct_count_query, conn)
    return count['distinct_count'][0]
distinct_count_dict = dict()
for table in table_names:
    distinct_count_dict[table[0]] = get_distinct_count_for_table(table[0], table[1])

In [12]:
distinct_count_df = pd.DataFrame.from_dict(distinct_count_dict, orient='index').reset_index()
distinct_count_df = distinct_count_df.rename(columns={"index": "table_name", 0: "distinct_count"})

In [13]:
print("Number of distinct rows in tables:")
distinct_count_df

Number of distinct rows in tables:


,table_name,distinct_count
0,artist_dim,9553
1,song_dim,14896
2,user_dim,96
3,time_dim,8023
4,songplay_fact,333


In [14]:
debug_query = f"""
SELECT user_id, count(*) as count
from user_dim
group by user_id
order by count desc
limit 10
"""
debug_results = sqlio.read_sql_query(debug_query, conn)
display(debug_results)

,user_id,count
0,15,2
1,16,2
2,29,2
3,36,2
4,49,2
5,80,2
6,85,2
7,88,2
8,2,1
9,3,1


In [15]:
debug_query = f"""
SELECT *
from user_dim
where user_id=15
limit 10
"""
debug_results = sqlio.read_sql_query(debug_query, conn)
display(debug_results)

,user_id,first_name,last_name,gender,level
0,15,Lily,Koch,F,paid
1,15,Lily,Koch,F,free


In [16]:
debug_query = f"""
SELECT artist_id, count(*) as count
from artist_dim
group by artist_id
order by count desc
limit 10
"""
debug_results = sqlio.read_sql_query(debug_query, conn)
display(debug_results)

,artist_id,count
0,AR065TW1187FB4C3A5,4
1,ARCBD0U1187FB466EF,4
2,ARMD3XX1187B9ACF84,4
3,ARS26BQ1187B99466D,4
4,ARTE9CG1187B99B1AF,4
5,ARV481W1187FB38CD9,4
6,ARZ5H0P1187B98A1DD,4
7,AR0CANF1187B9AF35F,3
8,AR0OQPL1187B9B549A,3
9,AR1OGXT1187B9893EB,3


In [17]:
debug_query = f"""
SELECT *
from artist_dim
where artist_id='AR065TW1187FB4C3A5'
limit 10
"""
debug_results = sqlio.read_sql_query(debug_query, conn)
display(debug_results)

,artist_id,name,location,latitude,longitude
0,AR065TW1187FB4C3A5,Tricky,"Knowle West, Bristol, Avon, Engla",NaN,NaN
1,AR065TW1187FB4C3A5,Tricky / The Mad Dog Reflex,"Knowle West, Bristol, Avon, Engla",51.4356,-2.57518
2,AR065TW1187FB4C3A5,Tricky,"Knowle West, Bristol, Avon, Engla",51.4356,-2.57518
3,AR065TW1187FB4C3A5,Nearly God,"Knowle West, Bristol, Avon, Engla",51.4356,-2.57518


**Get top 10 artist**

In [18]:
top_artists_query = f"""
SELECT artist_dim.name AS artist, COUNT(*) AS count 
FROM songplay_fact JOIN artist_dim ON songplay_fact.artist_id = artist_dim.artist_id
GROUP BY artist_dim.name
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_artists_query, conn)
display(result)

,artist,count
0,Dwight Yoakam,37
1,Kid Cudi,10
2,Lonnie Gordon,9
3,Ron Carter,9
4,B.o.B,8
5,Muse,6
6,Usher featuring Jermaine Dupri,6
7,Arctic Monkeys,5
8,Counting Crows,4
9,Metallica,4


**Get top 10 songs**

In [19]:
top_songs_query = f"""
SELECT song_dim.title AS song_name, COUNT(*) AS count 
FROM songplay_fact JOIN song_dim ON songplay_fact.song_id = song_dim.song_id
GROUP BY song_dim.title
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_songs_query, conn)
result

,song_name,count
0,You're The One,37
1,I CAN'T GET STARTED,9
2,Catch You Baby (Steve Pitron & Max Sanna Radio...,9
3,Nothin' On You [feat. Bruno Mars] (Album Version),8
4,Hey Daddy (Daddy's Home),6
5,Make Her Say,5
6,Up Up & Away,5
7,Unwell (Album Version),4
8,Mr. Jones,4
9,Supermassive Black Hole (Album Version),4


**What time do people listen to songs most**

In [20]:
top_hour_query = f"""
SELECT time_dim.hour AS hour_of_the_day, COUNT(*) AS count 
FROM songplay_fact JOIN time_dim ON songplay_fact.start_time = time_dim.start_time
GROUP BY time_dim.hour
ORDER BY count DESC
LIMIT 10
"""
result = sqlio.read_sql_query(top_hour_query, conn)
result

,hour_of_the_day,count
0,17,40
1,18,26
2,15,25
3,16,24
4,8,18
5,20,18
6,19,16
7,11,16
8,14,16
9,13,14


In [21]:
conn.close()